In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint

In [2]:
def lyap_spectrum_QR(Js,T,p = 1):
    K,n = Js.shape[0],Js.shape[-1]    
    old_Q = np.eye(n)
    H = np.eye(n)
    
    lexp  = np.zeros(n,dtype="float32")
    lexp_counts = np.zeros(lexp.shape)

    for t in range(K):
        H = Js[t] @ H 


        
        
        # QR-decomposition of T * old_Q
        mat_Q, mat_R = np.linalg.qr(np.dot(Js[t], old_Q))
        # force diagonal of R to be positive
        # (if QR = A then also QLL'R = A with L' = L^-1)
        sign_diag = np.sign(np.diag(mat_R))
        sign_diag[np.where(sign_diag == 0)] = 1
        sign_diag = np.diag(sign_diag)
        mat_Q = np.dot(mat_Q, sign_diag)
        mat_R = np.dot(sign_diag, mat_R)

        old_Q = mat_Q
        
        # successively build sum for Lyapunov exponents
        diag_R = np.diag(mat_R)
        
        # filter zeros in mat_R (would lead to -infs)
        idx = np.where(diag_R > 0)
        lexp_i = np.zeros(diag_R.shape, dtype="float32")
        lexp_i[idx] = np.log(diag_R[idx])
        lexp_i[np.where(diag_R == 0)] = np.inf

        lexp[idx] += lexp_i[idx]
        lexp_counts[idx] += 1


    # it may happen that all R-matrices contained zeros => exponent really has
    # to be -inf

    # normalize exponents over number of individual mat_Rs
    idx = np.where(lexp_counts > 0)
    #lexp[idx] /= lexp_counts[idx]
    lexp[np.where(lexp_counts == 0)] = np.inf

    lexp /= T
    
    return lexp

In [3]:
def lorenz(w,t, sigma, rho, beta):
    
    x,y,z = w[0],w[1],w[2]

    x_dot = sigma*(y - x)
    y_dot = x*(rho-z) - y
    z_dot = x*y - beta*z
    
    w_dot = [x_dot,y_dot,z_dot] 
    

    
    return w_dot



def get_lorenz_jacobian(sol,dt,sigma = 10, rho = 28,beta = 8/3):
    num_steps = sol.shape[0]
    Js = np.zeros((num_steps,3,3))
    I = np.eye(3)
    for t in range(num_steps):        
        Js[t,0,:] = [-sigma,sigma,0]
        Js[t,1,:] = [-sol[t,2]+ rho,-1,-sol[t,0]]
        Js[t,2,:] = [sol[t,1],sol[t,0],-beta]
        
        Js[t] = I + dt*Js[t]
        
    return Js



In [6]:
w0 = [.01,.01,.01]
T = 100 # s
dt = .01
K = int(T/dt)
t = np.linspace(0, T, K)
sol = odeint(lorenz, w0, t, args=(10, 28, 8/3))

Js = get_lorenz_jacobian(sol,dt)
lyaps = lyap_spectrum_QR(Js,T)
print(lyaps)

[  1.1438745    0.43619832 -15.7118    ]


In [6]:
sol.shape

(100000, 3)

In [7]:
T/dt

100000.0

In [9]:
Js.shape

(100000, 3, 3)